<a href="https://colab.research.google.com/github/Ravi1603/LegalInsight/blob/main/NLP_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers[sentencepiece] datasets sacrebleu rouge_score py7zr -q
from transformers import pipeline, set_seed

import matplotlib.pyplot as plt
from datasets import load_dataset
import pandas as pd
#!pip install --upgrade datasets
#from datasets import load_dataset, load_metric # This line should work now
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from transformers import Trainer, TrainingArguments
import sacrebleu

import nltk
from nltk.tokenize import sent_tokenize

from tqdm import tqdm
import torch

nltk.download("punkt")
!pip install evaluate
import evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.0/104.0 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.9/67.9 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 28.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 34.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.9/138.9 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.7/413.7 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.7 MB/s eta 0:00:00
   ━━━━━━

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 4.2 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


Govt report dataset

In [ ]:

# Load the dataset
dataset = load_dataset('ccdv/govreport-summarization')

# Display basic info
split_lengths = [len(dataset[split]) for split in dataset]
print(f"Split lengths: {split_lengths}")
print(f"Features: {dataset['train'].column_names}")
print("\nDialogue:")
print(dataset["test"][2]["report"])
print("\nSummary:")
print(dataset["test"][2]["summary"])

# Split the dataset into train, val, test sets
dataset = dataset['train'].train_test_split(test_size=0.2)
train_val_dataset = dataset['train'].train_test_split(test_size=0.1)
train_dataset = train_val_dataset['train']
val_dataset = train_val_dataset['test']
test_dataset = dataset['test']

# Print out the lengths of each split
print(f"Training set length: {len(train_dataset)}")
print(f"Validation set length: {len(val_dataset)}")
print(f"Test set length: {len(test_dataset)}")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/2.21k [00:00<?, ?B/s]

train-00000-of-00002.parquet:   0%|          | 0.00/228M [00:00<?, ?B/s]

train-00001-of-00002.parquet:   0%|          | 0.00/229M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/26.1M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/24.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/17517 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/973 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/973 [00:00<?, ? examples/s]

Split lengths: [17517, 973, 973]
Features: ['report', 'summary']

Dialogue:
According to the National Research Council, although the exact details cannot be predicted with certainty, climate change poses serious risks to many of the physical and ecological systems on which society depends. Moreover, according to key scientific assessments, the effects and costs of extreme weather events such as floods and droughts will increase in significance as what are considered rare events become more common and intense because of climate change. According to the National Academies of Sciences, Engineering, and Medicine, extreme weather events are directly traceable to loss of life, rising food and energy prices, increasing costs of disaster relief and insurance, fluctuations in property values, and concerns about national security. Table 1 shows seven effects commonly associated with climate change that DOD has documented. According to a 2010 National Research Council report on making informed de

legal dataset

In [ ]:
from datasets import load_dataset

# Load the new dataset
dataset = load_dataset('nhankins/legal_summary_data')

# Display basic info
split_lengths = [len(dataset[split]) for split in dataset]
print(f"Split lengths: {split_lengths}")
print(f"Features: {dataset['train'].column_names}")
print("\nSample Legal Document:")
print(dataset["test"][2]["original_text"])
print("\nSummary:")
print(dataset["test"][2]["reference_summary"])

# Split the dataset into train, val, test sets
dataset = dataset['train'].train_test_split(test_size=0.2)
train_val_dataset = dataset['train'].train_test_split(test_size=0.1)
train_dataset = train_val_dataset['train']
val_dataset = train_val_dataset['test']
test_dataset = dataset['test']

# Print out the lengths of each split
print(f"Training set length: {len(train_dataset)}")
print(f"Validation set length: {len(val_dataset)}")
print(f"Test set length: {len(test_dataset)}")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


legal_data.csv:   0%|          | 0.00/329k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/446 [00:00<?, ? examples/s]

Split lengths: [446]
Features: ['Unnamed: 0', 'doc', 'id', 'original_text', 'reference_summary', 'title']

Sample Legal Document:


KeyError: 'test'

In [ ]:
# Set device to CUDA if available, else CPU
device = "cuda" if torch.cuda.is_available() else "cpu"

Bart model

In [ ]:
from transformers import BartForConditionalGeneration, AutoTokenizer



# Load the BART model for conditional generation
model_bart = BartForConditionalGeneration.from_pretrained('facebook/bart-base').to(device)

# Load the tokenizer for the same BART model
tokenizer = AutoTokenizer.from_pretrained('facebook/bart-base')

# Now the model is loaded onto the correct device and the tokenizer is set up


In [ ]:

def generate_batch_sized_chunks(list_of_elements, batch_size):
    """split the dataset into smaller batches that we can process simultaneously
    Yield successive batch-sized chunks from list_of_elements."""
    for i in range(0, len(list_of_elements), batch_size):
        yield list_of_elements[i : i + batch_size]

govtreport

In [ ]:


def calculate_metric_on_test_ds(dataset, metric, model, tokenizer,
                                batch_size=16, device=device,
                                column_text="report",
                                column_summary="summary"):
    article_batches = list(generate_batch_sized_chunks(dataset[column_text], batch_size))
    target_batches = list(generate_batch_sized_chunks(dataset[column_summary], batch_size))

    for article_batch, target_batch in tqdm(
        zip(article_batches, target_batches), total=len(article_batches)):

        inputs = tokenizer(article_batch, max_length=1024, truncation=True,
                           padding="max_length", return_tensors="pt")

        summaries = model.generate(input_ids=inputs["input_ids"].to(device),
                                   attention_mask=inputs["attention_mask"].to(device),
                                   length_penalty=0.8, num_beams=8, max_length=128)
        ''' parameter for length penalty ensures that the model does not generate sequences that are too long. '''

        # Finally, we decode the generated texts, replace the token,
        # and add the decoded texts with the references to the metric.
        decoded_summaries = [tokenizer.decode(s, skip_special_tokens=True,
                                              clean_up_tokenization_spaces=True)
                             for s in summaries]

        decoded_summaries = [d.replace("", " ") for d in decoded_summaries]

        metric.add_batch(predictions=decoded_summaries, references=target_batch)

    # Finally compute and return the ROUGE scores.
    score = metric.compute()
    return score





Govt report

In [ ]:
pipe = pipeline('summarization', model=model_bart, tokenizer=tokenizer)
pipe_out = pipe(test_dataset[0]['report'][:1024])
print("\nPipeline Output:")
print(pipe_out)



Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.



Pipeline Output:
[{'summary_text': 'While federal agencies are beginning to recognize the need to adapt to climate change, there is a general lack of strategic coordination across agencies, and most efforts to Adapt to potential climate change impacts are preliminary. However, some states and localities have begun to make progress on adaptation independently and through partnerships with other entities, such as academic institutions. The subjects of our site visits in the United States—New York City; King County, Washington; and Maryland— have all begun to implement adaptive measures in sectors such as natural resource management and infrastructure. Their on-the-ground experiences can help inform the federal approach to adaptation, which is now'}]


Govt report scores

In [ ]:
rouge = evaluate.load('rouge')
predictions = [d['summary_text'] for d in pipe_out]  # Extract summary_text from the list of dictionaries
references = [dataset['test'][1]['summary']] # Wrap the reference string in a list

results = rouge.compute(predictions=predictions,
                         references=references,
                         use_aggregator=False)
print(results)

{'rouge1': [0.16310679611650486], 'rouge2': [0.011695906432748539], 'rougeL': [0.08543689320388351], 'rougeLsum': [0.08543689320388351]}


In [ ]:


# Summarization pipeline (replace `new_model` and `new_tokenizer` with your fine-tuned model)
pipe = pipeline('summarization', model=model_bart, tokenizer=tokenizer)

# Generate summaries and calculate ROUGE scores
rouge = evaluate.load('rouge')
num_samples = 100  # Adjust the number of test samples you want to evaluate

predictions = []
references = []

for i in range(num_samples):
    # Generate summary from the test report
    report_text = test_dataset[i]['report'][:1024]  # Ensure max input length is respected
    pipe_out = pipe(report_text)

    # Extract the generated summary
    predicted_summary = pipe_out[0]['summary_text']  # Adjust if different
    predictions.append(predicted_summary)

    # Get the reference summary from the dataset
    reference_summary = test_dataset[i]['summary']
    references.append(reference_summary)

# Compute ROUGE scores
results = rouge.compute(predictions=predictions,
                         references=references,
                         use_aggregator=True)  # Use aggregator to get overall scores

# Print out the results
print(f"ROUGE Scores: {results}")


Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


ROUGE Scores: {'rouge1': 0.23022501425820124, 'rouge2': 0.0816442304515473, 'rougeL': 0.14193789270021687, 'rougeLsum': 0.14241454949914306}


In [ ]:

# Load the BLEU metric
bleu = evaluate.load('sacrebleu')

# Assuming `pipe_out` is the output of your summarization pipeline
predictions = [d['summary_text'] for d in pipe_out]  # Extract summary_text from the list of dictionaries
references = [[dataset['test'][1]['summary']]]  # Wrap the reference string in a list of lists for BLEU

results = bleu.compute(predictions=predictions,
                         references=references)
print("\nBLEU Score:")
print(results)



BLEU Score:
{'score': 0.18301432285837685, 'counts': [47, 8, 1, 0], 'totals': [117, 116, 115, 114], 'precisions': [40.17094017094017, 6.896551724137931, 0.8695652173913043, 0.43859649122807015], 'bp': 0.05708306089007793, 'sys_len': 117, 'ref_len': 452}


In [ ]:
pipe = pipeline('summarization', model=model_bart, tokenizer=tokenizer)

num_samples = 100  # Adjust the number of test samples you want to evaluate
# Load the BLEU metric
bleu = evaluate.load('sacrebleu')
predictions = []
references = []

for i in range(num_samples):
    # Generate summary from the test report
    report_text = test_dataset[i]['report'][:1024]  # Ensure max input length is respected
    pipe_out = pipe(report_text)

    # Extract the generated summary
    predicted_summary = pipe_out[0]['summary_text']  # Adjust if different
    predictions.append(predicted_summary)

    # Get the reference summary from the dataset
    reference_summary = test_dataset[i]['summary']
    references.append(reference_summary)

results = bleu.compute(predictions=predictions,
                         references=references)
print("\nBLEU Score:")
print(results)

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.



BLEU Score:
{'score': 0.5951112818847709, 'counts': [6660, 2366, 1256, 907], 'totals': [11469, 11369, 11269, 11169], 'precisions': [58.06957886476589, 20.81097721875275, 11.145620729434732, 8.12069119885397], 'bp': 0.03272378885688946, 'sys_len': 11469, 'ref_len': 50689}


BART Training

In [ ]:
from transformers import DataCollatorForSeq2Seq

def tokenize_function(examples):
    # Tokenize the 'report' text (input to the model)
    model_inputs = tokenizer(examples['report'], max_length=1024, truncation=True)

    # Tokenize the 'summary' text (target for the model)
    labels = tokenizer(examples['summary'], max_length=150, truncation=True)

    # Add the tokenized summary as 'labels' without converting to numpy
    model_inputs['labels'] = labels['input_ids']

    return model_inputs

# Apply the tokenize function on the train and validation datasets
tokenized_train = train_dataset.map(tokenize_function, batched=True, batch_size=4)  # Reduce batch size here
tokenized_val = val_dataset.map(tokenize_function, batched=True, batch_size=4)

# Use DataCollator for Seq2Seq
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model_bart)

# Set the format for the tokenized datasets to include only necessary columns
tokenized_train.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
tokenized_val.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])


Map:   0%|          | 0/12611 [00:00<?, ? examples/s]

Map:   0%|          | 0/1402 [00:00<?, ? examples/s]

In [ ]:
# Define the Trainer with the correct arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=5,
    per_device_train_batch_size=8,  # Reduce batch size here
    per_device_eval_batch_size=8,   # Reduce batch size here
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy="epoch",
    save_steps=1000,
    save_total_limit=2,
    remove_unused_columns=True
)

# Initialize the Trainer
trainer = Trainer(
    model=model_bart,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    tokenizer=tokenizer,
    data_collator=data_collator
)

# Train the model
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/data/data_collator.py:656: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:278.)
  batch["labels"] = torch.tensor(batch["labels"], dtype=torch.int64)


Epoch,Training Loss,Validation Loss
1,2.200800,2.049290
2,1.995000,1.979100
3,1.910000,1.956868
4,1.842400,1.932557
5,1.691500,1.939138


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 128, 'min_length': 12, 'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 128, 'min_length': 12, 'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}
Some non-default generation parameters

TrainOutput(global_step=7885, training_loss=1.9847323320300503, metrics={'train_runtime': 10310.0358, 'train_samples_per_second': 6.116, 'train_steps_per_second': 0.765, 'total_flos': 3.84469323743232e+16, 'train_loss': 1.9847323320300503, 'epoch': 5.0})

In [ ]:
import os
import shutil
from google.colab import files

# Specify the directory
model_dir = '/content/drive/MyDrive/Colab Notebooks/NLP_trained_models/final_bart_model'

# Create the directory if it doesn't exist
os.makedirs(model_dir, exist_ok=True)

# Save the final model and tokenizer
trainer.save_model(model_dir)
tokenizer.save_pretrained(model_dir)

# Compress the model directory
shutil.make_archive(model_dir, 'zip', model_dir)

# Download the zip file
files.download(model_dir + '.zip')


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 128, 'min_length': 12, 'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
model_dir = './content/drive/MyDrive/Colab Notebooks/NLP_trained_models/final_bart_model'


In [ ]:
# Load the tokenizer
new_tokenizer = AutoTokenizer.from_pretrained(model_dir)

# Load the model
new_model = AutoModelForSeq2SeqLM.from_pretrained(model_dir).to('cuda' if torch.cuda.is_available() else 'cpu')


In [ ]:
pipe = pipeline('summarization', model=new_model, tokenizer=new_tokenizer)
pipe_out = pipe(test_dataset[1]['report'][:1024])
print("\nPipeline Output:")
print(pipe_out)



Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.



Pipeline Output:
[{'summary_text': 'Corporate credit unions occupy a unique niche among financial institutions. They are nonprofit financial cooperatives that are owned by natural person credit unions (that is, credit unions whose members are individuals), and provide lending, investment, and other financial services to these credit unions. For example, corporates offer loans to member credit unions, which in turn use these loans to meet the loan demands of their individual members. However, they are not the only financial institutions that provide products and services to credit unions that are similar to the correspo. GAO was asked to examine (1) the characteristics of corporate credit unions and (2)'}]


In [ ]:
rouge = evaluate.load('rouge')
predictions = [d['summary_text'] for d in pipe_out]  # Extract summary_text from the list of dictionaries
references = [dataset['test'][1]['summary']] # Wrap the reference string in a list

results = rouge.compute(predictions=predictions,
                         references=references,
                         use_aggregator=False)
print(results)

{'rouge1': [0.1669902912621359], 'rouge2': [0.0077972709551656924], 'rougeL': [0.08932038834951457], 'rougeLsum': [0.08932038834951457]}


In [ ]:


# Summarization pipeline (replace `new_model` and `new_tokenizer` with your fine-tuned model)
pipe = pipeline('summarization', model=new_model, tokenizer=new_tokenizer)

# Generate summaries and calculate ROUGE scores
rouge = evaluate.load('rouge')
num_samples = 100  # Adjust the number of test samples you want to evaluate

predictions = []
references = []

for i in range(num_samples):
    # Generate summary from the test report
    report_text = test_dataset[i]['report'][:1024]  # Ensure max input length is respected
    pipe_out = pipe(report_text)

    # Extract the generated summary
    predicted_summary = pipe_out[0]['summary_text']  # Adjust if different
    predictions.append(predicted_summary)

    # Get the reference summary from the dataset
    reference_summary = test_dataset[i]['summary']
    references.append(reference_summary)

# Compute ROUGE scores
results = rouge.compute(predictions=predictions,
                         references=references,
                         use_aggregator=True)  # Use aggregator to get overall scores

# Print out the results
print(f"ROUGE Scores: {results}")


Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


ROUGE Scores: {'rouge1': 0.23941080823644267, 'rouge2': 0.09248912032912118, 'rougeL': 0.14950241374825746, 'rougeLsum': 0.1494885420136337}


In [ ]:


# Load the BLEU metric
bleu = evaluate.load('sacrebleu')

# Assuming `pipe_out` is the output of your summarization pipeline
predictions = [d['summary_text'] for d in pipe_out]  # Extract summary_text from the list of dictionaries
references = [[dataset['test'][1]['summary']]]  # Wrap the reference string in a list of lists for BLEU

results = bleu.compute(predictions=predictions,
                         references=references)
print("\nBLEU Score:")
print(results)


BLEU Score:
{'score': 0.16118451859363928, 'counts': [46, 6, 1, 0], 'totals': [115, 114, 113, 112], 'precisions': [40.0, 5.2631578947368425, 0.8849557522123894, 0.44642857142857145], 'bp': 0.053373827087903364, 'sys_len': 115, 'ref_len': 452}


In [ ]:
pipe = pipeline('summarization', model=new_model, tokenizer=new_tokenizer)

num_samples = 100  # Adjust the number of test samples you want to evaluate
# Load the BLEU metric
bleu = evaluate.load('sacrebleu')
predictions = []
references = []

for i in range(num_samples):
    # Generate summary from the test report
    report_text = test_dataset[i]['report'][:1024]  # Ensure max input length is respected
    pipe_out = pipe(report_text)

    # Extract the generated summary
    predicted_summary = pipe_out[0]['summary_text']  # Adjust if different
    predictions.append(predicted_summary)

    # Get the reference summary from the dataset
    reference_summary = test_dataset[i]['summary']
    references.append(reference_summary)

results = bleu.compute(predictions=predictions,
                         references=references)
print("\nBLEU Score:")
print(results)

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.



BLEU Score:
{'score': 0.6591027516781269, 'counts': [6928, 2818, 1594, 1138], 'totals': [11246, 11146, 11046, 10946], 'precisions': [61.604125911435176, 25.282612596447155, 14.43056309976462, 10.396491869175955], 'bp': 0.029978000547341885, 'sys_len': 11246, 'ref_len': 50689}


Pegasus

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

model_ckpt = "google/pegasus-cnn_dailymail"

# Download the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model_bart = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt).to(device)

tokenizer_config.json:   0%|          | 0.00/88.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.12k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.91M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


generation_config.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

In [ ]:

# Run the pipeline
pipe = pipeline('summarization', model=model_ckpt)
pipe_out = pipe(dataset['test'][0]['report'][:1024])
print("\nPipeline Output:")
print(pipe_out)

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.



Pipeline Output:
[{'summary_text': 'SBA administers the 8(a) program, which includes determining whether it will accept requirements into the program .<n>In January 2012, we found that SBA lacked the data and insight to enforce some of the new requirements put in place to ensure that tribal 8(a) firms do not operate in the program in perpetuity .'}]


In [ ]:
rouge = evaluate.load('rouge')
predictions = [d['summary_text'] for d in pipe_out]  # Extract summary_text from the list of dictionaries
references = [dataset['test'][1]['summary']] # Wrap the reference string in a list

results = rouge.compute(predictions=predictions,
                         references=references,
                         use_aggregator=False)
print(results)

{'rouge1': [0.13855421686746988], 'rouge2': [0.012121212121212123], 'rougeL': [0.08433734939759036], 'rougeLsum': [0.08433734939759036]}


In [ ]:


# Summarization pipeline (replace `new_model` and `new_tokenizer` with your fine-tuned model)
pipe = pipeline('summarization', model=model_bart, tokenizer=tokenizer)

# Generate summaries and calculate ROUGE scores
rouge = evaluate.load('rouge')
num_samples = 100  # Adjust the number of test samples you want to evaluate

predictions = []
references = []

for i in range(num_samples):
    # Generate summary from the test report
    report_text = test_dataset[i]['report'][:1024]  # Ensure max input length is respected
    pipe_out = pipe(report_text)

    # Extract the generated summary
    predicted_summary = pipe_out[0]['summary_text']  # Adjust if different
    predictions.append(predicted_summary)

    # Get the reference summary from the dataset
    reference_summary = test_dataset[i]['summary']
    references.append(reference_summary)

# Compute ROUGE scores
results = rouge.compute(predictions=predictions,
                         references=references,
                         use_aggregator=True)  # Use aggregator to get overall scores

# Print out the results
print(f"ROUGE Scores: {results}")


Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


ROUGE Scores: {'rouge1': 0.12528832455345668, 'rouge2': 0.047796200127981026, 'rougeL': 0.08870271766531818, 'rougeLsum': 0.088998210618695}


In [ ]:

# Load the BLEU metric
bleu = evaluate.load('sacrebleu')

# Assuming `pipe_out` is the output of your summarization pipeline
predictions = [d['summary_text'] for d in pipe_out]  # Extract summary_text from the list of dictionaries
references = [[dataset['test'][1]['summary']]]  # Wrap the reference string in a list of lists for BLEU

results = bleu.compute(predictions=predictions,
                         references=references)
print("\nBLEU Score:")
print(results)



BLEU Score:
{'score': 0.059700979364122726, 'counts': [31, 2, 0, 0], 'totals': [65, 64, 63, 62], 'precisions': [47.69230769230769, 3.125, 0.7936507936507936, 0.4032258064516129], 'bp': 0.02271759863458736, 'sys_len': 65, 'ref_len': 311}


In [ ]:
pipe = pipeline('summarization', model=model_bart, tokenizer=tokenizer)

num_samples = 100  # Adjust the number of test samples you want to evaluate
# Load the BLEU metric
bleu = evaluate.load('sacrebleu')
predictions = []
references = []

for i in range(num_samples):
    # Generate summary from the test report
    report_text = test_dataset[i]['report'][:1024]  # Ensure max input length is respected
    pipe_out = pipe(report_text)

    # Extract the generated summary
    predicted_summary = pipe_out[0]['summary_text']  # Adjust if different
    predictions.append(predicted_summary)

    # Get the reference summary from the dataset
    reference_summary = test_dataset[i]['summary']
    references.append(reference_summary)

results = bleu.compute(predictions=predictions,
                         references=references)
print("\nBLEU Score:")
print(results)

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.



BLEU Score:
{'score': 0.0056270633123535726, 'counts': [3331, 1326, 828, 629], 'totals': [5684, 5584, 5484, 5384], 'precisions': [58.60309641097818, 23.74641833810888, 15.098468271334792, 11.682763744427934], 'bp': 0.00025280316736326817, 'sys_len': 5684, 'ref_len': 52764}


T5

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

model_ckpt = "google/flan-t5-base"

# Download the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model_bart = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt).to(device)

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:

# Run the pipeline
pipe = pipeline('summarization', model=model_bart)
pipe_out = pipe(dataset['test'][0]['report'][:1024])
print("\nPipeline Output:")
print(pipe_out)

Exception: Impossible to guess which tokenizer to use. Please provide a PreTrainedTokenizer class or a path/identifier to a pretrained tokenizer.

In [ ]:
rouge = evaluate.load('rouge')
predictions = [d['summary_text'] for d in pipe_out]  # Extract summary_text from the list of dictionaries
references = [dataset['test'][1]['summary']] # Wrap the reference string in a list

results = rouge.compute(predictions=predictions,
                         references=references,
                         use_aggregator=False)
print(results)

{'rouge1': [0.03296703296703297], 'rouge2': [0.0], 'rougeL': [0.030219780219780223], 'rougeLsum': [0.030219780219780223]}


In [ ]:
# Summarization pipeline (replace `model_bart` and `tokenizer` with your fine-tuned model)
pipe = pipeline('summarization', model=model_bart, tokenizer=tokenizer, device=device)

# Generate summaries and calculate ROUGE scores
rouge = evaluate.load('rouge')
num_samples = 100  # Adjust the number of test samples you want to evaluate

predictions = []
references = []

for i in range(num_samples):
    # Get the report text (ensure max input length is respected)
    report_text = test_dataset[i]['report'][:1024]

    # Dynamically set the max_length for summarization based on input length
    input_length = len(report_text.split())  # Tokenize the input and calculate its length
    max_summary_length = int(input_length * 0.5)  # Set summary length to 50% of the input length (or adjust this factor)

    # Generate summary from the test report with a dynamic max_length
    pipe_out = pipe(report_text, max_length=max_summary_length)

    # Extract the generated summary
    predicted_summary = pipe_out[0]['summary_text']
    predictions.append(predicted_summary)

    # Get the reference summary from the dataset
    reference_summary = test_dataset[i]['summary']
    references.append(reference_summary)

# Compute ROUGE scores
results = rouge.compute(predictions=predictions, references=references, use_aggregator=True)

# Print out the results
print(f"ROUGE Scores: {results}")


ROUGE Scores: {'rouge1': 0.0854198836088197, 'rouge2': 0.03686787382265838, 'rougeL': 0.06736106473823489, 'rougeLsum': 0.06751913539234371}


In [ ]:
pipe = pipeline('summarization', model=model_bart, tokenizer=tokenizer, device=device)

num_samples = 100  # Adjust the number of test samples you want to evaluate
# Load the BLEU metric
bleu = evaluate.load('sacrebleu')
predictions = []
references = []

for i in range(num_samples):
    # Generate summary from the test report
    report_text = test_dataset[i]['report'][:1024]  # Ensure max input length is respected
    pipe_out = pipe(report_text,max_length=max_summary_length)

    # Extract the generated summary
    predicted_summary = pipe_out[0]['summary_text']  # Adjust if different
    predictions.append(predicted_summary)

    # Get the reference summary from the dataset
    reference_summary = test_dataset[i]['summary']
    references.append(reference_summary)

results = bleu.compute(predictions=predictions,
                         references=references)
print("\nBLEU Score:")
print(results)


BLEU Score:
{'score': 3.0758231726604376e-06, 'counts': [2185, 893, 507, 368], 'totals': [3262, 3162, 3062, 2962], 'precisions': [66.98344573881054, 28.241619228336496, 16.557805355976488, 12.424037812288994], 'bp': 1.2314894925723868e-07, 'sys_len': 3262, 'ref_len': 55160}
